<a href="https://colab.research.google.com/github/Martoto/BigData_ForecastHackathon/blob/main/silicionauta.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import requests

file_path = '/content/hackathon_2025_templates.zip'
file_id = '1Ed-nqUyCT0z-T0AvHwLMsGqHYqjw2248' # This is the file ID

if not os.path.exists(file_path):
    print(f'{file_path} not found. Downloading...')

    # Construct the direct download URL
    download_url = f'https://drive.google.com/uc?export=download&id={file_id}'

    try:
        response = requests.get(download_url, stream=True)
        response.raise_for_status() # Raise an exception for bad status codes

        with open(file_path, 'wb') as f:
            for chunk in response.iter_content(chunk_size=8192):
                f.write(chunk)

        print(f'{file_path} downloaded successfully.')

    except requests.exceptions.RequestException as e:
        print(f"Error downloading the file: {e}")
        print("Please ensure the file is shared publicly or with 'Anyone with the link'.")

else:
    print(f'{file_path} found.')

/content/hackathon_2025_templates.zip found.
